## Recurrent Neural Networks

In this assignment, we will learn about recurrent neural networks. We will create an RNN and learn to classify text data.

In [0]:
# Install tf2.0
!pip install --upgrade tensorflow --quiet

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 450kB 38.4MB/s 
     |████████████████████████████████| 3.8MB 50.1MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.11.0 which is incompatible.


In [0]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

In [0]:
yelp = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/yelp_labeled.csv', error_bad_lines=False)

b'Skipping line 281: expected 2 fields, saw 3\nSkipping line 290: expected 2 fields, saw 3\nSkipping line 296: expected 2 fields, saw 3\nSkipping line 322: expected 2 fields, saw 3\nSkipping line 373: expected 2 fields, saw 3\nSkipping line 417: expected 2 fields, saw 3\nSkipping line 427: expected 2 fields, saw 3\nSkipping line 429: expected 2 fields, saw 3\nSkipping line 577: expected 2 fields, saw 3\nSkipping line 578: expected 2 fields, saw 3\nSkipping line 611: expected 2 fields, saw 3\nSkipping line 677: expected 2 fields, saw 3\nSkipping line 771: expected 2 fields, saw 3\nSkipping line 930: expected 2 fields, saw 3\nSkipping line 979: expected 2 fields, saw 4\nSkipping line 980: expected 2 fields, saw 3\n'


In [0]:
yelp.head()

,text,sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [0]:
yelp.sentiment.value_counts()

1    494
0    482
Name: sentiment, dtype: int64

We have loaded a Yelp review dataset above. A positive sentiment is classified as 1 and a negative sentiment is classified as 1. 

In [0]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

In [0]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In the code block above, we have functions to remove stopwords, stem, and normalize the text (remove special characters and trim white space). Apply the normalize function to every yelp review and assign the normalized text to a new column.

In [0]:
# Answer below:
yelp['proc_text'] = yelp['text'].apply(normalize)

In [0]:
yelp.head()

,text,sentiment,proc_text
0,Wow... Loved this place.,1,wow love place
1,Crust is not good.,0,crust not good
2,Not tasty and the texture was just nasty.,0,not tasti textur nasti
3,Stopped by during the late May bank holiday of...,1,stop late may bank holiday rick steve recommen...
4,The selection on the menu was great and so wer...,1,select menu great price


Next, use the one hot function for text encoding and encode the normalized text. Determine the vocabulary size to perform the encoding.

In [0]:
# Answer below:
from tensorflow.keras.preprocessing.text import one_hot


In [0]:
vocab_size = len(set(yelp.proc_text.str.split().sum()))
encoded = [one_hot(l, vocab_size) for l in list(yelp['proc_text'])]

Convert the encoded sequences into a numpy array and make sure all reviews are the same length using the `pad_sequences` function in Keras.

In [0]:
# Answer below:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [0]:
data=pad_sequences(encoded )

Split the data into train and test. Use 20% for test. The sentiment column should be used as the target variable.

In [0]:
# Answer below:
from sklearn.model_selection import train_test_split
X = data
y = yelp.sentiment.values

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size=0.2, random_state=84)

Create a sequential model. The model should contain an embedding layer with input dim that is the size of the largest encoding in the vocabulary. The output dim should be 100, the input length is the number of columns in the training data. 
After the embedding layer, add a SimpleRNN layer with unit size 32, a dense layer of size 8 and a dense output layer.

In [0]:
def convert_data(data,step):
    X,y = [], []
    for i in range(len(data)-step):
        X.append(data[i:d],1)
        y.append(data[d,])
    return np.array(X), np.array(y)

In [0]:
#input_dim = max of len(yelp.text)
output_dim = 100
inpu_length = 1

In [0]:
step = 3

In [0]:
# Answer below:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Embedding
model = Sequential()
model.add(Embedding(input_dim=np.max(X_train)+1,output_dim=X_train.shape[1]))
model.add(SimpleRNN(units=32, input_shape=(1,step),activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 83)          135124    
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 32)                3712      
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 264       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 9         
Total params: 139,109
Trainable params: 139,109
Non-trainable params: 0
_________________________________________________________________


Compile using the optimizer of your choice, use MSE for your loss function. Fit the model using a batch size of 128 and 50 epochs

In [0]:
np.max(X_train)

1627

In [0]:
# Answer below:
model.compile(loss='binary_crossentropy',optimizer='rmsprop', metrics=['accuracy'])
# Fit the model: setting verbose=1 prints out some results after each epoch
model.fit(X_train, y_train, batch_size=128, epochs=50, verbose=2, validation_data=(X_test, y_test))
# Score the model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 780 samples, validate on 196 samples
Epoch 1/50
780/780 - 1s - loss: 0.4792 - accuracy: 0.9244 - val_loss: 1.0693 - val_accuracy: 0.7296
Epoch 2/50
780/780 - 0s - loss: 0.0084 - accuracy: 0.9974 - val_loss: 1.0901 - val_accuracy: 0.7296
Epoch 3/50
780/780 - 0s - loss: 0.0079 - accuracy: 0.9974 - val_loss: 1.1219 - val_accuracy: 0.7245
Epoch 4/50
780/780 - 0s - loss: 0.0075 - accuracy: 0.9974 - val_loss: 1.1445 - val_accuracy: 0.7296
Epoch 5/50
780/780 - 0s - loss: 0.0075 - accuracy: 0.9974 - val_loss: 1.1731 - val_accuracy: 0.7245
Epoch 6/50
780/780 - 0s - loss: 0.0068 - accuracy: 0.9974 - val_loss: 1.1903 - val_accuracy: 0.7296
Epoch 7/50
780/780 - 0s - loss: 0.0068 - accuracy: 0.9974 - val_loss: 1.2117 - val_accuracy: 0.7296
Epoch 8/50
780/780 - 0s - loss: 0.0067 - accuracy: 0.9974 - val_loss: 1.2688 - val_accuracy: 0.7092
Epoch 9/50
780/780 - 0s - loss: 0.0066 - accuracy: 0.9974 - val_loss: 1.3013 - val_accuracy: 0.7143
Epoch 10/50
780/780 - 0s - loss: 0.0067 - accuracy: 0.

In [0]:
from sklearn.model_selection import train_test_split as tts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

In [0]:
X_train, X_test, y_train, y_test = tts(yelp['proc_text'],y,test_size=0.2)

In [0]:
vect = TfidfVectorizer()

In [0]:
x_train_vect = vect.fit_transform(X_train)
x_test_vect = vect.transform(X_test)

In [0]:
rfc = RandomForestClassifier(random_state=42, n_estimators=100)
rfc.fit(x_train_vect, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [0]:
rfc.score(x_test_vect,y_test)

0.75

In [0]:
rfc.score(x_train_vect,y_train)

1.0